In [1]:
import numpy as np
import pandas as pd
from sklearn import *


/home/dcek/anaconda2/envs/py27/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/dcek/anaconda2/envs/py27/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/home/dcek/anaconda2/envs/py27/lib/python2.7/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module

In [121]:

train = pd.read_csv('train_de.csv')
test = pd.read_csv('test.csv')
COMMENT = 'comment_text'
train[COMMENT].fillna("unknown", inplace=True)
test[COMMENT].fillna("unknown", inplace=True)


In [62]:
coly = [c for c in train.columns if c not in ['id','comment_text']]
y = train[coly]
tid = test['id'].values

df = pd.concat([train['comment_text'], test['comment_text']], axis=0)
df = df.fillna("unknown")
nrow = train.shape[0]
print(nrow)
tfidf = feature_extraction.text.TfidfVectorizer(stop_words='english', max_features=800000)
data = tfidf.fit_transform(df)


159571


In [63]:
def compute_auc(y_true, y_pred):
  try:
    return metrics.roc_auc_score(y_true, y_pred)
  except ValueError:
    return np.nan

In [122]:
def train_model(train_x, train_y, val_x, val_y, text_x, fold_id):
    label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
    preds = np.zeros((text_x.shape[0], len(label_cols)))
    meta = np.zeros((val_x.shape[0], len(label_cols)))
    #for i, j in enumerate(label_cols):
    #    print('fit', j)
    #label_train = pd.DataFrame(train_y, columns = label_cols)
    model = ensemble.ExtraTreesClassifier(n_jobs=-1, random_state=3)
    model.fit(train_x, train_y)
    meta = model.predict_proba(val_x)
    auc = compute_auc(val_y, meta)
    preds = model.predict_proba(text_x)
    #meta[:,i] = m.best_estimator_.predict_proba(val_x.multiply(r))[:,1]
    #preds[:,i] = m.best_estimator_.predict_proba(test_x.multiply(r))[:,1]
    print("fold {} finish".format(fold_id))
    return preds, meta, auc

In [123]:
test = tfidf.transform(test['comment_text'])
X = train['comment_text']
y = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]
total_auc = []
fold_size = len(X) // 10
test_predicts_list = []
total_meta = []
for fold_id in range(0, 10):
    fold_start = fold_size * fold_id
    fold_end = fold_start + fold_size
            
    if fold_id == 10 - 1:
        fold_end = len(X)

    train_x = np.concatenate([X[:fold_start], X[fold_end:]])
    train_y = np.concatenate([y[:fold_start], y[fold_end:]])
    #print(train_y.shape)
    val_x = X[fold_start:fold_end]
    val_y = y[fold_start:fold_end]
    
    #train_x = pd.DataFrame(train_x, columns = ['comment_text'])   
    #val_x = pd.DataFrame(val_x, columns = ['comment_text'])
    train_x = tfidf.transform(train_x)
    val_x = tfidf.transform(val_x)
    print(train_x.shape)
    
    test_predicts, meta, auc = train_model(train_x, train_y, val_x, val_y, test, fold_id)
    total_auc.append(auc)
    test_predicts_list.append(test_predicts)
    #if (fold_id == 0):
    #    total_meta = meta
    #else:
    total_meta.append(meta)
    print("finish fold {}".format(fold_id))

(143614, 339279)
fold 0 finish
finish fold 0
(143614, 339279)
fold 1 finish
finish fold 1
(143614, 339279)
fold 2 finish
finish fold 2
(143614, 339279)
fold 3 finish
finish fold 3
(143614, 339279)
fold 4 finish
finish fold 4
(143614, 339279)
fold 5 finish
finish fold 5
(143614, 339279)
fold 6 finish
finish fold 6
(143614, 339279)
fold 7 finish
finish fold 7
(143614, 339279)
fold 8 finish
finish fold 8
(143613, 339279)
fold 9 finish
finish fold 9


In [124]:
for auc in total_auc:
    print(auc)

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [55]:
total_meta[0]

[array([[0.9, 0.1],
        [1. , 0. ],
        [0.9, 0.1],
        ...,
        [1. , 0. ],
        [1. , 0. ],
        [0.2, 0.8]]), array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]]), array([[1. , 0. ],
        [1. , 0. ],
        [1. , 0. ],
        ...,
        [1. , 0. ],
        [1. , 0. ],
        [0.8, 0.2]]), array([[1. , 0. ],
        [1. , 0. ],
        [0.9, 0.1],
        ...,
        [1. , 0. ],
        [1. , 0. ],
        [1. , 0. ]]), array([[1. , 0. ],
        [1. , 0. ],
        [1. , 0. ],
        ...,
        [1. , 0. ],
        [1. , 0. ],
        [0.8, 0.2]]), array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]])]

In [125]:
final_meta = []
shapes = []
i = 0
for meta in total_meta:
    sub2 = meta
    sub2 = pd.DataFrame([[c[1] for c in sub2[row]] for row in range(len(sub2))]).T
    display(sub2.head(n=10))
    shapes.append(sub2.shape)
    if (i==0):
        final_meta = sub2.values
    else:
        final_meta = np.concatenate((final_meta, sub2.values), axis=0)
        
    i += 1


,0,1,2,3,4,5
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.2,0.0,0.1,0.0,0.1,0.0
3,0.1,0.0,0.0,0.0,0.1,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0
6,1.0,1.0,1.0,0.0,1.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0


,0,1,2,3,4,5
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.1,0.0,0.0,0.0,0.0,0.0
3,0.2,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0
5,0.1,0.0,0.0,0.0,0.1,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0
8,0.3,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0


,0,1,2,3,4,5
0,0.000000,0.00000,0.000000,0.0,0.000000,0.00000
1,0.000000,0.00000,0.000000,0.0,0.000000,0.00000
2,0.000000,0.00000,0.000000,0.0,0.000000,0.00000
3,0.100000,0.00000,0.100000,0.0,0.100000,0.00000
4,0.006863,0.00098,0.003922,0.0,0.001961,0.00098
5,0.050000,0.00000,0.000000,0.0,0.050000,0.00000
6,0.300000,0.00000,0.000000,0.0,0.000000,0.00000
7,0.000000,0.00000,0.000000,0.0,0.000000,0.00000
8,0.000000,0.00000,0.000000,0.0,0.000000,0.00000
9,0.200000,0.00000,0.000000,0.0,0.000000,0.00000


,0,1,2,3,4,5
0,0.100000,0.00000,0.00000,0.0,0.00000,0.00000
1,0.000000,0.00000,0.00000,0.0,0.00000,0.00000
2,0.000000,0.00000,0.00000,0.0,0.00000,0.00000
3,0.207921,0.00099,0.20396,0.0,0.00297,0.00198
4,0.000000,0.00000,0.00000,0.0,0.00000,0.00000
5,0.000000,0.00000,0.00000,0.0,0.00000,0.00000
6,0.000000,0.00000,0.00000,0.0,0.00000,0.00000
7,0.000000,0.00000,0.00000,0.0,0.00000,0.00000
8,0.000000,0.00000,0.00000,0.0,0.00000,0.00000
9,0.000000,0.00000,0.00000,0.0,0.00000,0.00000


,0,1,2,3,4,5
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0
5,0.3,0.0,0.1,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0
8,1.0,0.0,0.1,0.0,0.1,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0


,0,1,2,3,4,5
0,0.000000,0.00000,0.000000,0.0,0.000000,0.000000
1,0.100000,0.00000,0.100000,0.0,0.100000,0.000000
2,0.300000,0.00000,0.100000,0.0,0.100000,0.100000
3,0.000000,0.00000,0.000000,0.0,0.000000,0.000000
4,0.000000,0.00000,0.000000,0.0,0.000000,0.000000
5,0.000000,0.00000,0.000000,0.0,0.000000,0.000000
6,0.700000,0.10000,0.600000,0.0,0.600000,0.100000
7,0.050000,0.00000,0.000000,0.0,0.050000,0.000000
8,0.008824,0.00098,0.004902,0.0,0.002941,0.001961
9,0.000000,0.00000,0.000000,0.0,0.000000,0.000000


,0,1,2,3,4,5
0,0.100000,0.000000,0.000000,0.0,0.000000,0.000000
1,0.008571,0.000952,0.004762,0.0,0.002857,0.001905
2,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
3,0.100000,0.000000,0.000000,0.0,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
7,0.100000,0.000000,0.100000,0.0,0.100000,0.000000
8,0.100000,0.000000,0.000000,0.0,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


,0,1,2,3,4,5
0,0.9,0.0,0.0,0.0,0.9,0.9
1,0.1,0.1,0.1,0.0,0.1,0.0
2,0.1,0.0,0.0,0.0,0.0,0.0
3,0.1,0.0,0.0,0.0,0.0,0.0
4,0.1,0.0,0.1,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0
6,0.8,0.4,0.8,0.0,0.8,0.1
7,0.0,0.0,0.0,0.0,0.0,0.0
8,0.2,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0


,0,1,2,3,4,5
0,0.0,0.0,0.1,0.0,0.0,0.0
1,0.1,0.0,0.0,0.0,0.1,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,0.1,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0
7,0.1,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0


,0,1,2,3,4,5
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.1,0.0,0.1,0.0,0.0,0.0
2,0.3,0.0,0.1,0.0,0.2,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0
5,0.1,0.0,0.1,0.0,0.1,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0


In [97]:
for s in shapes:
    print(s)

(15957, 6)
(15957, 6)
(15957, 6)
(15957, 6)
(15957, 6)
(15957, 6)
(15957, 6)
(15957, 6)
(15957, 6)
(15958, 6)


In [111]:
final_meta.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.2,0.0,0.1,0.0,0.1,0.0
3,0.1,0.0,0.0,0.0,0.1,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0


In [126]:
final_meta = pd.DataFrame(final_meta, columns = label_cols)

In [127]:
auc = compute_auc(y.values, final_meta.values)

In [128]:
print(auc)

0.8725931530929013


In [105]:
y.shape

(159571, 6)

In [106]:
final_meta.shape

(159571, 6)

In [31]:
hehe = np.array(test_predicts_list[0])

In [32]:
hehe.shape

(6, 153164, 2)

In [35]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
subm = pd.read_csv('sample_submission.csv')
submid = pd.DataFrame({'id': subm["id"]})
submission = pd.concat([submid, pd.DataFrame(test_predicts_list[0], columns = label_cols)], axis=1)

ValueError: Must pass 2-d input

In [30]:
test_predicts = np.ones(test_predicts_list[0].shape)

AttributeError: 'list' object has no attribute 'shape'

In [28]:
hi.shape

(10, 6, 153164, 2)

In [129]:
test_predicts_listed = []
for pred in test_predicts_list:
    sub2 = pred
    sub2 = pd.DataFrame([[c[1] for c in sub2[row]] for row in range(len(sub2))]).T
    display(sub2.head(n=10))
    test_predicts_listed.append(sub2.values)

,0,1,2,3,4,5
0,0.500000,0.0,0.300000,0.0,0.200000,0.000000
1,0.000000,0.0,0.000000,0.0,0.000000,0.000000
2,0.020588,0.0,0.011765,0.0,0.005882,0.002941
3,0.000000,0.0,0.000000,0.0,0.000000,0.000000
4,0.000000,0.0,0.000000,0.0,0.000000,0.000000
5,0.100000,0.0,0.000000,0.0,0.000000,0.000000
6,0.000000,0.0,0.000000,0.0,0.000000,0.000000
7,0.000000,0.0,0.000000,0.0,0.000000,0.000000
8,0.100000,0.0,0.100000,0.0,0.000000,0.000000
9,0.000000,0.0,0.000000,0.0,0.000000,0.000000


,0,1,2,3,4,5
0,0.9,0.4,0.8,0.0,0.8,0.2
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0
6,0.1,0.0,0.1,0.0,0.1,0.0
7,0.2,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0


,0,1,2,3,4,5
0,0.8,0.2,0.7,0.0,0.7,0.1
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0
8,0.1,0.0,0.1,0.0,0.2,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0


,0,1,2,3,4,5
0,0.600000,0.10000,0.600000,0.0,0.400000,0.100000
1,0.000000,0.00000,0.000000,0.0,0.000000,0.000000
2,0.023762,0.00297,0.011881,0.0,0.008911,0.005941
3,0.000000,0.00000,0.000000,0.0,0.000000,0.000000
4,0.000000,0.00000,0.000000,0.0,0.000000,0.000000
5,0.000000,0.00000,0.000000,0.0,0.000000,0.000000
6,0.100000,0.00000,0.100000,0.0,0.000000,0.000000
7,0.000000,0.00000,0.000000,0.0,0.000000,0.000000
8,0.300000,0.00000,0.100000,0.0,0.100000,0.000000
9,0.000000,0.00000,0.000000,0.0,0.000000,0.000000


,0,1,2,3,4,5
0,0.600000,0.00000,0.600000,0.0,0.400000,0.100000
1,0.000000,0.00000,0.000000,0.0,0.000000,0.000000
2,0.006863,0.00098,0.002941,0.0,0.001961,0.001961
3,0.000000,0.00000,0.000000,0.0,0.000000,0.000000
4,0.000000,0.00000,0.000000,0.0,0.000000,0.000000
5,0.000000,0.00000,0.000000,0.0,0.000000,0.000000
6,0.100000,0.00000,0.100000,0.0,0.000000,0.000000
7,0.000000,0.00000,0.000000,0.0,0.000000,0.000000
8,0.100000,0.00000,0.000000,0.0,0.100000,0.100000
9,0.000000,0.00000,0.000000,0.0,0.000000,0.000000


,0,1,2,3,4,5
0,0.700000,0.20000,0.400000,0.0,0.400000,0.000000
1,0.100000,0.00000,0.000000,0.0,0.000000,0.000000
2,0.008824,0.00098,0.004902,0.0,0.002941,0.001961
3,0.000000,0.00000,0.000000,0.0,0.000000,0.000000
4,0.008824,0.00098,0.004902,0.0,0.002941,0.001961
5,0.000000,0.00000,0.000000,0.0,0.000000,0.000000
6,0.100000,0.00000,0.000000,0.0,0.000000,0.000000
7,0.000000,0.00000,0.000000,0.0,0.000000,0.000000
8,0.000000,0.00000,0.000000,0.0,0.000000,0.000000
9,0.000000,0.00000,0.000000,0.0,0.000000,0.000000


,0,1,2,3,4,5
0,0.500000,0.000000,0.400000,0.1,0.300000,0.200000
1,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
2,0.008571,0.000952,0.004762,0.0,0.002857,0.001905
3,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
4,0.100000,0.000000,0.100000,0.0,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
7,0.100000,0.000000,0.000000,0.0,0.000000,0.000000
8,0.100000,0.000000,0.000000,0.0,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


,0,1,2,3,4,5
0,0.600000,0.000000,0.400000,0.0,0.400000,0.200000
1,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
2,0.026214,0.002913,0.014563,0.0,0.008738,0.005825
3,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
7,0.100000,0.000000,0.100000,0.0,0.100000,0.000000
8,0.100000,0.000000,0.100000,0.0,0.100000,0.000000
9,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


,0,1,2,3,4,5
0,0.800000,0.100000,0.700000,0.1,0.800000,0.100000
1,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
2,0.007692,0.000962,0.004808,0.0,0.002885,0.001923
3,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
8,0.200000,0.000000,0.100000,0.0,0.100000,0.000000
9,0.100000,0.000000,0.000000,0.0,0.100000,0.000000


,0,1,2,3,4,5
0,0.6,0.1,0.5,0.1,0.4,0.2
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0
7,0.1,0.0,0.0,0.0,0.0,0.0
8,0.2,0.0,0.1,0.0,0.1,0.1
9,0.0,0.0,0.0,0.0,0.0,0.0


In [82]:
test_predicts = np.ones(test_predicts_listed[0].shape)
for fold_predict in test_predicts_listed:
    test_predicts *= fold_predict

test_predicts **= (1. / len(test_predicts_listed))

In [83]:
test_predicts.shape

(153164, 6)

In [85]:
test.shape

(153164, 339279)

In [53]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
subm = pd.read_csv('sample_submission.csv')
submid = pd.DataFrame({'id': subm["id"]})
submission = pd.concat([submid, pd.DataFrame(test_predicts, columns = label_cols)], axis=1)
submission.to_csv('ET_ori_pred_CV.csv', index=False)
submission.head(n=10)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.808565,0.0,0.572922,0.0,0.471613,0.0
1,0000247867823ef7,0.000000,0.0,0.000000,0.0,0.000000,0.0
2,00013b17ad220c46,0.000000,0.0,0.000000,0.0,0.000000,0.0
3,00017563c3f7919a,0.000000,0.0,0.000000,0.0,0.000000,0.0
4,00017695ad8997eb,0.000000,0.0,0.000000,0.0,0.000000,0.0
5,0001ea8717f6de06,0.000000,0.0,0.000000,0.0,0.000000,0.0
6,00024115d4cbde0f,0.000000,0.0,0.000000,0.0,0.000000,0.0
7,000247e83dcc1211,0.000000,0.0,0.000000,0.0,0.000000,0.0
8,00025358d4737918,0.000000,0.0,0.000000,0.0,0.000000,0.0
9,00026d1092fe71cc,0.000000,0.0,0.000000,0.0,0.000000,0.0


In [108]:
subm = pd.read_csv('sample_submission.csv')
submid = pd.DataFrame({'id': subm["id"]})
total_meta_data = pd.concat([submid, pd.DataFrame(final_meta, columns = label_cols)], axis=1)
total_meta_data.to_csv('meta_ET_ori_cv.csv', index=False)
display(total_meta_data.head(n=20))


,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.0,0.0,0.0,0.0,0.0,0.0
1,0000247867823ef7,0.0,0.0,0.0,0.0,0.0,0.0
2,00013b17ad220c46,0.2,0.0,0.1,0.0,0.1,0.0
3,00017563c3f7919a,0.1,0.0,0.0,0.0,0.1,0.0
4,00017695ad8997eb,0.0,0.0,0.0,0.0,0.0,0.0
5,0001ea8717f6de06,0.0,0.0,0.0,0.0,0.0,0.0
6,00024115d4cbde0f,1.0,1.0,1.0,0.0,1.0,0.0
7,000247e83dcc1211,0.0,0.0,0.0,0.0,0.0,0.0
8,00025358d4737918,0.0,0.0,0.0,0.0,0.0,0.0
9,00026d1092fe71cc,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:

sub2.columns = [x+'_' if x not in ['id'] else x for x in sub2.columns]
blend = pd.merge(sub1, sub2, how='left', on='id')
for c in coly:
    blend[c] = blend[c] * 0.8 + blend[c+'_'] * 0.2
    blend[c] = blend[c].clip(0+1e12, 1-1e12)
blend = blend[sub1.columns]
blend.to_csv('submission.csv', index=False)

In [44]:

coly = [c for c in train.columns if c not in ['id','comment_text']]
y = train[coly]
tid = test['id'].values

df = pd.concat([train['comment_text'], test['comment_text']], axis=0)
df = df.fillna("unknown")
nrow = train.shape[0]

tfidf = feature_extraction.text.TfidfVectorizer(stop_words='english', max_features=800000)
data = tfidf.fit_transform(df)

model = ensemble.ExtraTreesClassifier(n_jobs=-1, random_state=3)
model.fit(data[:nrow], y)
print(1- model.score(data[:nrow], y))
sub2 = model.predict_proba(data[nrow:])
sub2 = pd.DataFrame([[c[1] for c in sub2[row]] for row in range(len(sub2))]).T
sub2.columns = coly
sub2['id'] = tid
for c in coly:
    sub2[c] = sub2[c].clip(0+1e12, 1-1e12)

sub2.columns = [x+'_' if x not in ['id'] else x for x in sub2.columns]


0.001284694587362356


In [45]:
sub2.head(n=10)

,toxic_,severe_toxic_,obscene_,threat_,insult_,identity_hate_,id
0,0.700000,0.000000,0.500000,0.0,0.300000,0.100000,00001cee341fdb12
1,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0000247867823ef7
2,0.011475,0.001639,0.006557,0.0,0.003279,0.003279,00013b17ad220c46
3,0.100000,0.000000,0.000000,0.0,0.000000,0.000000,00017563c3f7919a
4,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,00017695ad8997eb
5,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0001ea8717f6de06
6,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,00024115d4cbde0f
7,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,000247e83dcc1211
8,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,00025358d4737918
9,0.100000,0.000000,0.000000,0.0,0.000000,0.000000,00026d1092fe71cc


In [ ]:
blend = pd.merge(sub1, sub2, how='left', on='id')
for c in coly:
    blend[c] = blend[c] * 0.8 + blend[c+'_'] * 0.2
    blend[c] = blend[c].clip(0+1e12, 1-1e12)
blend = blend[sub1.columns]
blend.to_csv('submission.csv', index=False)